In [1]:
from ews.models import *
import pandas as pd
from fastkml import  kml

Package `lxml` missing. Pretty print will be disabled


In [4]:
kml_file = "D:/DWC_plattform/rain.kml"
with open(kml_file, 'rt', encoding="utf-8") as myfile:
    doc=myfile.read()

In [6]:
k = kml.KML()

In [7]:
k.from_string(doc)

In [8]:
k

In [12]:
features = list(k.features())

In [14]:
len(features)

1

In [27]:
list(list(k.features())[0].features())

In [28]:
rain_raw = list(list(k.features())[0].features())

In [29]:
# print out the first 5 stations just as a sanity check
for s in rain_raw[:5]:
    # a type of pygeoif.geometry.Point
    point = s._geometry.geometry
    print(s.name, point.x, point.y)

RB (93) 2.4736346 48.8836453
LG (93) 2.5342468 48.9201319
BR (93) 2.3371865 48.9460012
BG (93) 2.424896 48.877997
DR (93) 2.4517002 48.9249472


In [31]:
# lets put them into pandas dataframes
import pandas as pd

rain_list = [{
    'name': s.name,
    'lon': s._geometry.geometry.x,
    'lat': s._geometry.geometry.y
} for s in rain_raw]

rain = pd.DataFrame(rain_list)

# again, let's sanity check we have the right stuff
print(rain.head())



      name       lon        lat
0  RB (93)  2.473635  48.883645
1  LG (93)  2.534247  48.920132
2  BR (93)  2.337187  48.946001
3  BG (93)  2.424896  48.877997
4  DR (93)  2.451700  48.924947


In [41]:
l = rain.name.str.split(' ')

In [42]:
l

0       [RB, (93)]
1       [LG, (93)]
2       [BR, (93)]
3       [BG, (93)]
4       [DR, (93)]
          ...     
63    [PL02, (75)]
64    [PL16, (75)]
65    [PL01, (75)]
66    [PL06, (75)]
67    [PL07, (75)]
Name: name, Length: 68, dtype: object

In [45]:
rain.name = [item[0] for item in l]

In [59]:
rainvalues = pd.read_csv("D:/DigitalWaterCity/data_processed/rain_siaap.csv", parse_dates = True, index_col = "Date")

In [67]:
rv = rainvalues.groupby("gauge").resample("D").sum()

In [68]:
print(rv.head())

                          value
gauge         Date             
BONN38_P (mm) 2016-01-01   0.24
              2016-01-02   0.04
              2016-01-03   0.56
              2016-01-04   0.00
              2016-01-05   0.04


In [72]:
rv = rv.reset_index()

In [80]:
gn = rv.gauge.str.split("_")

In [81]:
rv.gauge= [item[0] for item in gn]

In [89]:
FeatureType.objects.all()

<QuerySet [<FeatureType: WWTP>, <FeatureType: Network>, <FeatureType: River>]>

In [85]:
rv.gauge.unique()

array(['BONN38', 'CHAM23', 'CHAM40'], dtype=object)

In [86]:
rain.name.unique()

array(['RB', 'LG', 'BR', 'BG', 'DR', 'NM', 'DU', 'CL', 'PN', 'LV', 'NC',
       'BM', 'NP', 'NE', 'PM', 'MT', 'MO', 'OU', 'TF', 'BB', 'SD', 'RL',
       'JE', 'GY', 'VI', 'GP', 'SL', 'LHAY35', 'MAIS37', 'MAIS32',
       'VITR22', 'IVRY01', 'IVRY16', 'CHAM40', 'QUEU09', 'SMAN31',
       'FONT05', 'NOIS17', 'SUCY29', 'BONN38', 'VALE39', 'ORLY20',
       'VALE06', 'BOIS25', 'MAND30', 'SANT08', 'SMFO26', 'JOIN10',
       'VILL14', 'CACH15', 'FRES34', 'RUNG19', 'CHAM23', 'VITR02', 'PL12',
       'PL11', 'PL10', 'PL09', 'PL13', 'PL14', 'PL03', 'PL04', 'PL05',
       'PL02', 'PL16', 'PL01', 'PL06', 'PL07'], dtype=object)

In [90]:
f = FeatureType()

In [91]:
f.name = "Rainfall"

In [92]:
f.unit = "mm"

In [93]:
f.save()

In [94]:
rain

,name,lon,lat
0,RB,2.473635,48.883645
1,LG,2.534247,48.920132
2,BR,2.337187,48.946001
3,BG,2.424896,48.877997
4,DR,2.451700,48.924947
...,...,...,...
63,PL02,2.322609,48.839926
64,PL16,2.332222,48.829983
65,PL01,2.288032,48.852728
66,PL06,2.256447,48.839511


In [95]:
for index, row in rain.iterrows():
    loc = Site()
    loc.name = row['name']
    loc.ref_name = row['name']
    loc.geom = {"coordinates": [row['lon'],row['lat']], "type": "Point"}
    loc.feature_type = f
    loc.owner = User.objects.get(id = 1)
    try:
        loc.save()
    except:
        print(loc)

In [96]:
rain

,name,lon,lat
0,RB,2.473635,48.883645
1,LG,2.534247,48.920132
2,BR,2.337187,48.946001
3,BG,2.424896,48.877997
4,DR,2.451700,48.924947
...,...,...,...
63,PL02,2.322609,48.839926
64,PL16,2.332222,48.829983
65,PL01,2.288032,48.852728
66,PL06,2.256447,48.839511
